### 네이버 쇼핑 리뷰 감성 분석

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
# warnings.filterwarnings('ignore')

In [10]:
url = 'https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt'
df = pd.read_table(url, names=['score', 'review'])
df.head()

,score,review
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [11]:
# score 의 분포
df.score.value_counts()

score
5    81177
2    63989
1    36048
4    18786
Name: count, dtype: int64

In [12]:
# 평점이 4, 5 점은 긍정(1), 나머지는 부정(0)
df.score = df.score.apply(lambda x: 1 if x>=4 else 0)
df.head()

,score,review
0,1,배공빠르고 굿
1,0,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,1,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,0,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,1,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


##### 2. 데이터 전처리

In [13]:
# 결측치 확인
df.isna().sum()

score     0
review    0
dtype: int64

In [14]:
# 중복 데이터 확인
print(df.shape)
print(df.review.nunique())

(200000, 2)
199908


In [15]:
# 중복 데이터 제거
df.drop_duplicates(subset=['review'], inplace=True)
df.shape

(199908, 2)

In [16]:
# 한글 이외의 데이터는 제거
# df.review = df.review.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', regex=True).str.strip()
df.review = df.review.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', regex=True)

In [17]:
# 한글 이외의 데이터를 제거함으로써 발생하는 ''를 제거
df.review.replace('', np.nan, inplace=True)
df.isna().sum()

score     0
review    0
dtype: int64

##### 3. 한글 형태소 분석 및 불용어 제거

In [18]:
with open('../data/한글불용어.txt') as st:
    lines = st.readlines()
stop_words = [line.split('\t')[0] for line in lines]

In [19]:
from konlpy.tag import Okt
okt = Okt()

In [21]:
%%time
reviews = []
for review in df.review:
    morphs = okt.morphs(review, stem=True)
    tmp = [word for word in morphs  if word not in stop_words]
    reviews.append(' '.join(tmp))

CPU times: total: 10min 37s
Wall time: 10min 39s


##### 4. 데이터셋 분리

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    reviews, df.score.values, stratify=df.score.values, random_state=2023 
)

##### 5. 피쳐 변환 + 모델 학습/평가

In [25]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

In [26]:
pipeline = Pipeline([
    ('tvect', TfidfVectorizer(ngram_range=(1, 2), max_df=0.95)),
    ('nb', MultinomialNB())

])
%time pipeline.fit(X_train, y_train)

CPU times: total: 8.27 s
Wall time: 7.3 s


Pipeline(steps=[('tvect', TfidfVectorizer(max_df=0.95, ngram_range=(1, 2))),
                ('nb', MultinomialNB())])

In [27]:
pipeline.score(X_test, y_test)

0.8915701222562379

In [29]:
from sklearn.linear_model import LogisticRegression
pipe2 = Pipeline([
    ('tvect', TfidfVectorizer(ngram_range=(1, 2), max_df=0.95)),
    ('lr', LogisticRegression(max_iter=500, random_state=2023))
])
%time pipe2.fit(X_train, y_train)

CPU times: total: 26.7 s
Wall time: 24.2 s


Pipeline(steps=[('tvect', TfidfVectorizer(max_df=0.95, ngram_range=(1, 2))),
                ('lr', LogisticRegression(max_iter=500, random_state=2023))])

In [30]:
pipe2.score(X_test, y_test)

0.8962922944554496